# Yelp Data Merging

In [1]:
# Copied and pasted from my imports from project 3, 
# delete if we dont need(Aerika)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB, BernoulliNB

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords 

import regex as re

from scipy.stats import ttest_ind

In [2]:
file_paths = []
for file in os.listdir('../data'):
    if 'businesses2019' in file:
        file_paths.append('../data/'+file)

In [3]:
master_df = {
    'id': [],
    'latitude': [],
    'longitude': [],
    'price': [],
    'review_count': [],
    'rating': [],
    'zip_code': [],
    'city': [],
    'alias': [],
    'category': [],
}

master_df = pd.DataFrame(master_df)

In [4]:
for path in file_paths:
    
    master_df = pd.concat([master_df, pd.read_csv(path)])

master_df.head()

C:\Users\m6zha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,nelsons-rancho-palos-verdes,"['comfortfood', 'bars']",Rancho Palos Verdes,6cj1SyuhiBcuw5O0zLooUw,33.736753,-118.399057,$$,3.5,1034.0,90275
1,cielo-point-rancho-palos-verdes,['tradamerican'],Rancho Palos Verdes,zBCW_Ozlk5VeSoue626JUg,33.737931,-118.396190,$$,4.0,16.0,90275
2,bashi-rancho-palos-verdes,"['asianfusion', 'sushi', 'noodles']",Rancho Palos Verdes,49Ib3q8XKr6nnWIxO_qx0Q,33.738209,-118.397316,$$,3.5,205.0,90275
3,spa-cafe-rancho-palos-verdes-3,['cafes'],Rancho Palos Verdes,XxE8ccFye-bSfx4N8rJ8RA,33.738226,-118.397658,$$,3.0,5.0,90275
4,lobby-bar-rancho-palos-verdes,"['newamerican', 'lounges']",Rancho Palos Verdes,T9h21w-CPJi9NyLLIl8vTQ,33.738267,-118.397911,$$,4.0,74.0,90275


In [5]:
# Seeing shape of dataframe BEFORE dropping duplicates 

master_df.shape

(98830, 10)

In [6]:
# Dropping duplicates

master_df = master_df.sort_values(['alias',
                       'id', 'latitude', 'longitude', 'review_count'],
                      ascending = False).drop_duplicates(subset = ['id', 'alias', 'zip_code'],
                                                         keep ='first')

In [7]:
# Shape of df AFTER dropping duplicates

master_df.shape

(34876, 10)

In [8]:
# Resetting master_df index

master_df.reset_index(inplace=True)
master_df.drop(columns='index', inplace=True)

# NaNs in master_df

In [9]:
master_df.isna().sum()

alias             1
category          1
city              1
id                1
latitude         17
longitude        17
price             1
rating            1
review_count      1
zip_code        123
dtype: int64

In [10]:
# Dropping rows w/ prices that aren't $, $$, $$$, or $$$$.

data_drop_df = master_df.loc[(master_df['price'] != '$') & (master_df['price'] != '$$') & (master_df['price'] != '$$$') & (master_df['price'] != '$$$$')]
drop_indexes = list(data_drop_df.index)

master_df.drop(index=drop_indexes, inplace = True)
master_df.shape


(34704, 10)

In [11]:
# Businesses that don't have latitude/zipcodes = Food Trucks, Caterers --> dropped

food_trucks = master_df.loc[(master_df['latitude'].isna()) | (master_df['zip_code'].isna())].index.tolist()

master_df.drop(index = food_trucks, inplace = True)
master_df.shape


(34588, 10)

In [12]:
master_df.isna().sum()

alias           0
category        0
city            0
id              0
latitude        0
longitude       0
price           0
rating          0
review_count    0
zip_code        0
dtype: int64

# Replacing \\$ in price to numbers

In [13]:
master_df['price'] = master_df['price'].map({'$$$$':4, '$$$':3, '$$':2,'$':1})

In [14]:
master_df.head(2)

,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,重庆特色小面-chongqing-special-noodles-san-gabriel-3,"['noodles', 'chinese']",San Gabriel,mm6l24khOfXhKqn4pKVojA,34.103070,-118.091840,1,4.0,204.0,91776
1,重庆小面-best-noodle-house-rosemead-3,"['noodles', 'szechuan']",Rosemead,DgXxBgUEvARu45x7RSgYRw,34.081182,-118.066226,1,4.0,141.0,91770


In [15]:
master_df.loc[master_df['latitude']>]

SyntaxError: invalid syntax (<ipython-input-15-cf44d25830d2>, line 1)

# Dropping Rows 
- convert zip_codes from float -> int -> str
- zipcodes that don't start with '9'

In [16]:
master_df = master_df.reset_index(drop=True)

In [17]:
master_df['zip_code'] = master_df['zip_code'].astype(int).astype(str)

In [18]:
notin_la = []
for i in range(len(master_df['zip_code'])):
    if master_df['zip_code'][i].startswith('9') == False:
        notin_la.append(master_df['zip_code'].index[i])

In [19]:
notin_la

[1144,
 2080,
 3699,
 5544,
 7982,
 9716,
 11556,
 11557,
 11902,
 11911,
 11922,
 11927,
 11929,
 11938,
 11939,
 11941,
 11952,
 11964,
 11965,
 12004,
 12025,
 12026,
 12027,
 12053,
 12057,
 12063,
 12066,
 12067,
 12068,
 12069,
 12070,
 12085,
 12086,
 12089,
 12092,
 12100,
 17671,
 17941,
 18246,
 18247,
 19896,
 19897,
 22479,
 28930,
 30545,
 32529]

In [20]:
master_df.shape

(34588, 10)

In [21]:
master_df = master_df.drop(index=notin_la).reset_index(drop=True)

# Removing rows that aren't around the same latitude/longitude

In [22]:
master_df = master_df.drop(index=master_df.loc[(master_df['longitude'] > -117) | (master_df['latitude'] > 35)].index).reset_index(drop=True)

In [23]:
master_df.shape

(34530, 10)

# Feature Engineering

In [24]:
master_df['price*rating'] = master_df['price'] * master_df['rating']

## Save the cleaned data to a CSV

In [27]:
# Don't uncomment this unless you actually want to create a new csv

# master_df.to_csv('../data/master.csv')